In [1]:
import kwikfile as kf
import numpy as np
import imp
%matplotlib inline
import matplotlib.pyplot as plt
from klusta.kwik import KwikModel
import matplotlib as mpl
from pylab import *
import sys

import ipywidgets as widgets
from IPython.display import display
import time

class HeatMap:
    """!  @brief model that returns an average detection of each site of a given kwik file.
    
    @author: Harison P B de Oliveira
    @date: 2018.May.23
    """
    
    def __init__(self,kpath=None,name=None):
        if kpath is None:
            print ("It still with no path")
        else:
            self.kwik_model=KwikModel(kpath)
            self.kwikfile=kf.KwikFile(kpath)
            self.kpath=kpath
            if name is None:
                self.name=self.kwikfile.name
            else:
                self.name=name
            print ("Created class on = %s !" % self.kpath)
    
    def cluster_amplitude(self,cluster_id=None):
        """!  @brief Returns an average amplitude of each site of a cluster.
    
        @author: Harison P B de Oliveira
        @date: 2018.May.23
        """
        
        if cluster_id is not None:
            clusters = self.kwik_model.spike_clusters
            idx=np.argwhere (clusters==cluster_id)
            w=self.kwik_model.all_waveforms[idx]
            return w
        else:
            print("Enter the cluster valid!")
            return None
        
        
    
    def group_amplitude(self,group_name=None):
        """!  @brief Returns an average amplitude of each site of a group.
    
        @author: Harison P B de Oliveira
        @date: 2018.May.23
        """
        
        if group_name is None:
            print("Enter the group name.")
            return None
        else:
            if group_name is "mua" or "good" or "noise":
                progress = widgets.IntProgress(min=0, description='Loading clusters:',max=len(self.kwikfile.clusters(group_name=group_name)))
                display(progress)
                
                median_amplitude_group=np.zeros((10,32))
                
                for cluster_id in self.kwikfile.clusters(group_name=group_name):
                    print("1")
                    w=self.cluster_amplitude(cluster_id)
                    num_channels = w.shape[2]
                    print("2")
                    for ch in range (0,num_channels):
                        #print(ch)
                        median_amplitude_group[ch,:]+=np.mean(w[:,:,ch],0)
                        median_amplitude_group[ch,:]/=2
                    print("3")
                    progress.value += 1
                return median_amplitude_group
            else:
                print("Enter the group name valid!")
                return None
                    
        def plot_amplitude(self,median_amplitude_group=None):
            """!  @brief Returns an image with maximum average amplitude of each site of a shank.
    
            @author: Harison P B de Oliveira
            @date: 2018.May.24
            """
            
            if median_amplitude_group is None:
                return None
            else:
                fig,ax = plt.subplots(figsize=(6, 10))
                num_channels=median_amplitude_group.shape(0)
                max_amplitude=np.zeros((num_channels))
                
                for ch in range(0,num_channels):
                    top=max(median_amplitude_group[ch,:])
                    low=min(median_amplitude_group[ch,:])
                    max_amplitude[ch]=top+low
                        
                posx=np.flipud (self.kwik_model.channel_positions [:,0])
                posy=np.flipud (self.kwik_model.channel_positions [:,1])
                print(posx,posy)
                cmap = mpl.cm.jet
                norm = mpl.colors.Normalize(vmin=min(max_amplitude), vmax=max(max_amplitude))
                cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                        norm=norm,
                                        extend='both',
                                        extendfrac='auto',
                                        spacing='uniform',
                                        orientation='vertical') 
                a=list()
                dx=35
                for ch in range (0,num_channels):
                    if ch%2!=0:
                        x_offset = posx[ch]+dx
                    else:
                        x_offset = posx[ch]-dx
                    y_offset =posy[ch]*2
                    circle = plt.Circle((x_offset,y_offset),10,color=cb1.to_rgba(max_amplitude[ch]))
                    a.append(circle)
                
                ax.cla()
                ax.set_xlim((min(posx)-dx-10, max(posx)+dx+10))
                ax.set_ylim((min(posy)*2-10, max(posy)*2+10))
                for i in range(0,num_channels):
                    c=a[i]
                    ax.add_artist(c)
                    
                data = np.clip(randn(0, 0), -1, 1)
                cax = ax.imshow(data, interpolation='None', cmap=cm.jet)
                cbar = fig.colorbar(cax,ticks=[0,0.5,1],orientation='vertical')
                cbar.ax.set_yticklabels(['Low','Medium','High'])
                plt.axis('off')
        
                plt.show()  
                

In [2]:
from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider, Text
import ipywidgets as widgets
import time

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center'
)

data = [
    Box([Label(value='Path'),
            Text("/home/harison/UFPE/data/g01.kwik")
            ], layout=form_item_layout),
    Box([Label(value='Group name:'),
         Dropdown(options=['good', 'mua', 'noise'])],layout=form_item_layout)
    
]

form = Box(data, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))
display(form)

button = widgets.Button(description="Enviar informações!")
display(button)

def on_button_clicked(b):
    #kpath=form_items[0].children[0].value
    t0=time.time()
    print(t0)
    kpath=data[0].children[1].value
    group_name=data[1].children[1].value
    hm = HeatMap(kpath)
    ampl=hm.group_amplitude(group_name)
    print(time.time()-t0)
    #hm.amplitudes_heat_map(ampl)
    #group_name=
    #print("%s: %s" %(children[0].description,children[0].value))
    
    
button.on_click(on_button_clicked)

Box(children=(Box(children=(Label(value='Path'), Text(value='/home/harison/UFPE/data/g01.kwik')), layout=Layout(display='flex', flex_flow='row', justify_content='center')), Box(children=(Label(value='Group name:'), Dropdown(options=('good', 'mua', 'noise'), value='good')), layout=Layout(display='flex', flex_flow='row', justify_content='center'))), layout=Layout(align_items='stretch', border='solid 2px', display='flex', flex_flow='column', width='50%'))

Button(description='Enviar informações!', style=ButtonStyle())

1527791177.5547392
Created class on = /home/harison/UFPE/data/g01.kwik !
Created class on = /home/harison/UFPE/data/g01.kwik !


IntProgress(value=0, description='Loading clusters:', max=25)

1
2
3
3
1
